**Caso 5**

# Caso de Estudio: Edificios Inteligentes y Energía

## Contexto del Negocio

En la construcción de edificios sostenibles (como los de certificación LEED), **saber cuánta energía gastarán es la clave del éxito**. No se trata solo de ecología, sino de estrategia:

* **Ahorro:** Menos gasto en recibos de luz y gas a largo plazo.
* **Equipos:** Saber exactamente qué tamaño de aire acondicionado o calefacción comprar.
* **Normas:** Cumplir con las leyes ambientales y reducir la contaminación.

La energía que necesita un edificio para estar a una temperatura agradable depende de su **forma y diseño**: si es muy alto, si tiene muchas ventanas, qué tan grande es el techo o hacia dónde está orientado. Predecir esto **antes de construir** permite elegir el mejor diseño y ahorrar mucho dinero.


## Objetivo del Caso de Estudio

El objetivo es **analizar la eficiencia de los edificios** usando datos de sus características físicas para tomar mejores decisiones.

### Tareas Principales

1. **Predicción de Gasto (Regresión):**
Calcular el número exacto de **carga de calefacción** (`Y1`). Esto nos dice cuánta energía consumirá el edificio para mantenerse caliente.
2. **Etiquetado de Diseño (Clasificación):**
Separar los diseños en dos grupos: **Eficiente (1)** o **No Eficiente (0)**. Si el gasto de energía supera un límite, el diseño se descarta por no ser ahorrador.


In [16]:
# IMPORTACION DE LIBRERIAS
# Sin estas librerías no podemos leer datos ni trabajar con ellos.
import pandas as pd
import numpy as np
import requests
import zipfile
import io

In [17]:
# URL directa del dataset (UCI)
# Descargamos los datos por internet, así todos usan exactamente los mismos datos, sin errores.
url = "https://archive.ics.uci.edu/static/public/242/energy%2Befficiency.zip"

response = requests.get(url)
response.status_code

200

In [18]:
# INSPECCION DEL ARCHIVO ZIP.
# Se abre el archivo y se lee el dataset.
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.namelist()

In [19]:
# APERTURA DEL ARCHIVO
# Abre el archivo Excel y lo convierte en una tabla de datos.
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("ENB2012_data.xlsx") as f:
        df = pd.read_excel(f)

df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [20]:
# REVISION DE LAS FILAS Y COLUMNAS
df.shape

(768, 10)

In [21]:
# REVISAR EL TIPO DE DATOS CON LOS QUE TRABAJAMOS
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


In [22]:
# REVISAMOS SI EXISTEN VALORE FALTANTES EN CADA COLUMNA
df.isna().sum()

,0
X1,0
X2,0
X3,0
X4,0
X5,0
X6,0
X7,0
X8,0
Y1,0
Y2,0


In [23]:
# RENOMBRAMOS LAS COLUMNAS
# Cambiamos por sus nombres que vimos en la url
df.columns = [
    "Relative_Compactness",      # X1
    "Surface_Area",              # X2
    "Wall_Area",                 # X3
    "Roof_Area",                 # X4
    "Overall_Height",            # X5
    "Orientation",               # X6
    "Glazing_Area",              # X7
    "Glazing_Area_Distribution", # X8
    "Heating_Load",              # Y1
    "Cooling_Load"               # Y2
]
df.head()

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [24]:
# FEATURE ENGIENEERING (Overall_Surface)
# Creamos una nueva columna sumando paredes + techo.
df["Overall_Surface"] = df["Wall_Area"] + df["Roof_Area"]

df[["Wall_Area", "Roof_Area", "Overall_Surface"]].head()
# Overall_Surface = Wall_Area + Roof_Area captura la superficie total de intercambio térmico,
# simplificando y clarificando la relación con la carga térmica.

,Wall_Area,Roof_Area,Overall_Surface
0,294.0,110.25,404.25
1,294.0,110.25,404.25
2,294.0,110.25,404.25
3,294.0,110.25,404.25
4,318.5,122.50,441.00


In [25]:
# DEFINIMOS LAS VARIABLES FISICAS
# Elegimos qué columnas se van a usar como entrada del modelo. Esto para decirle al modelo que informacion puede ver
physical_features = [
    "Relative_Compactness",
    "Surface_Area",
    "Wall_Area",
    "Roof_Area",
    "Overall_Height",
    "Orientation",
    "Glazing_Area",
    "Glazing_Area_Distribution",
    "Overall_Surface"
]

target = "Heating_Load"

In [26]:
# CONSTRUCCION DEL PIPELINE
# Creacion de una “receta” automática para preparar los datos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [27]:
# APLICACION DEL PIPELINE
# Escala los datos para que todos estén en el mismo tamaño.
physical_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler())
    ]
)

In [28]:
X = df[physical_features]
y = df[target]

X_scaled = physical_pipeline.fit_transform(X)

In [29]:
# VERIFICACION DEL ESCALADO
# Comprobacion de que el escalado funcionó bien.
X_scaled.mean(axis=0)

array([-7.40148683e-17, -4.16333634e-16,  0.00000000e+00,  2.17418676e-16,
        0.00000000e+00,  0.00000000e+00,  1.48029737e-16,  0.00000000e+00,
       -4.07081776e-16])

In [30]:
X_scaled.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Validación Cruzada y Comparación de Modelos

En esta sección, comparamos dos modelos de regresión regularizada: **Ridge** y **Lasso**. 
La regularización es fundamental para evitar el sobreajuste (overfitting), especialmente cuando tenemos variables que podrían estar correlacionadas.

*   **Ridge (L2):** Penaliza los coeficientes al cuadrado, tendiendo a reducirlos pero manteniéndolos todos en el modelo.
*   **Lasso (L1):** Penaliza el valor absoluto de los coeficientes, lo que puede llevar a algunos a ser exactamente cero, realizando una selección de variables automática.

Utilizaremos **Validación Cruzada de 10 iteraciones (10-fold Cross-Validation)** para obtener una estimación robusta del error (RMSE).

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, KFold

Ridge RMSE: 2.9311 ± 0.2583
Lasso RMSE: 3.4336 ± 0.2578


In [ ]:
# Definición de los modelos con parámetros por defecto
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=1.0)

In [ ]:
# Configuración de Validación Cruzada (10 folds)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# Función para calcular RMSE en validación cruzada
def get_cv_rmse(model, X, y):
    # neg_mean_squared_error devuelve valores negativos, los pasamos a positivos y sacamos raíz
    mse_scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf)
    rmse_scores = np.sqrt(-mse_scores)
    return rmse_scores

In [ ]:
# Ejecución de validación cruzada
ridge_rmse = get_cv_rmse(ridge_model, X_scaled, y)
lasso_rmse = get_cv_rmse(lasso_model, X_scaled, y)

In [ ]:
# Reporte de resultados
print(f"Ridge RMSE: {ridge_rmse.mean():.4f} ± {ridge_rmse.std():.4f}")
print(f"Lasso RMSE: {lasso_rmse.mean():.4f} ± {lasso_rmse.std():.4f}")

### Justificación del Procedimiento

1.  **Métrica RMSE:** El Error Cuadrático Medio Raíz (RMSE) se expresa en las mismas unidades que la variable objetivo (`Heating_Load`), lo que facilita la interpretación del error promedio del modelo.
2.  **K-Fold (K=10):** Dividir el dataset en 10 partes asegura que cada dato se use tanto para entrenamiento como para validación, reduciendo el sesgo en la evaluación del rendimiento.
3.  **Comparación:** Ridge suele funcionar mejor cuando muchas variables contribuyen un poco, mientras que Lasso es preferible si sospechamos que solo unas pocas variables son realmente importantes.